# Evaluate model

In [1]:
from dataset import DwgDataset
from sketch_rnn import Trainer
from IPython.display import clear_output
from plot_graphics import plot_history
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
import PIL
import imageio
import glob
import os

Could not import drawSvg


In [2]:

dwg_dataset = DwgDataset(pickle_file='test_dataset_groups.pickle', batch_size=1, limit_seq_len=500)
trainer = Trainer(dwg_dataset, preload_model=True)

trainer.encoder.eval()
trainer.decoder.eval()


   ClassName                               GroupId  XLine1Point.X  \
0       Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
1       Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
2       Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
3       Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
6       Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
7       Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
8       Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
9       Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
10      Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
12      Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
15      Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
16      Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
17      Line  0335ebcf-8848-4c70-8a97-04195d5816b9            NaN   
18      Line  0335ebcf-8848-4c70-8

C:\Users\mihanick.MCAD\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.45 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


DecoderRNN(
  (fc_hc): Linear(in_features=128, out_features=2048, bias=True)
  (lstm): LSTM(133, 1024, dropout=0.45)
  (fc_params): Linear(in_features=1024, out_features=123, bias=True)
)

In [3]:
# use imageio to create animated gif using images 

def create_gif_from_frames(folder):
    '''
    creates gif file from png frames in folder
    frames should be named 'gen_frame*.png'
    png frames are erased in process
    
    uses
     imageio
     os
     glob
    '''

    anim_file = folder+ '/predictions.gif'

    with imageio.get_writer(anim_file, mode = 'I', fps=0.75) as writer:
        filenames = glob.glob(folder+'/gen_frame*.png')
        filenames = sorted(filenames)

        last = -1

        for i, filename in enumerate(filenames):
            #frame = 2*(i**0.5)
            #if (round(frame)> round(last)):
            #    last = frame
            #else:
            #    continue

            image = imageio.imread(filename)
            writer.append_data(image)
        return anim_file

In [4]:
torch.set_printoptions(profile="full")
def plot_generated_stroke_over_sequence(sequence, stroke=None, batch_number=0):
    print(sequence.shape, stroke)
    
    if stroke is not None:
        sequence = torch.cat([sequence.detach().cpu(), stroke.unsqueeze(0)], dim=0)
        
    sequence = sequence[:,batch_number,:].cpu().numpy()
    sequence = sequence[sequence[:, 4] < 1]
    # print(sequence)
    x_sample = 100*np.cumsum(sequence[:,0], 0)
    y_sample = 100*np.cumsum(sequence[:,1], 0)
    p1 = np.array(sequence[:,2])
    p2 = np.array(sequence[:,3])
    p3 = np.array(sequence[:,4])
    
    sequence = np.stack([x_sample,y_sample,p1,p2,p3], axis=1)
    strokes = np.split(sequence, np.where(sequence[:,2]>0)[0]+1)
    
    fig = plt.figure()
    for i in range(len(strokes)):
        s=strokes[i]
        cl = 'black'
        if i == len(strokes) - 1:
            cl='red'
        plt.plot(s[:,0],-s[:,1], color=cl)
        
    return fig

import shutil
folder = 'img_g'

if os.path.exists(folder):
    shutil.rmtree(folder)
os.mkdir(folder)

_, b = next(enumerate(dwg_dataset.train_loader))
input_seq, mask = b
input_seq = input_seq.to(trainer.device).transpose(0,1)

for i in range(1, trainer.max_seq_length):
    
    # print(i)
    seq_so_far = torch.zeros_like(input_seq)
    seq_so_far[:i+1,:,:] = input_seq[:i+1,:,: ]

    if seq_so_far[i,0,4]==1:
        continue

    #print(i, seq_so_far[:10])
    #print(seq_so_far.shape)
    z, _, _ = trainer.encoder(seq_so_far)

    # expand z in order it to be able to concatenate with inputs
    z_stack = torch.stack([z] * (trainer.max_seq_length))

    # inputs is the concatination of z and batch_inputs:
    inputs = torch.cat([seq_so_far, z_stack], 2)

    pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy, q, _, _ = trainer.decoder(inputs, z)

    generated_stroke = None #trainer.sample_next_state(pi, q, mu_x, mu_y, sigma_x, sigma_y, rho_xy)
    
    #clear_output()
    graph = plot_generated_stroke_over_sequence(seq_so_far, generated_stroke, 0)
    #display(graph)
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()
    pil_image = PIL.Image.frombytes('RGB', canvas.get_width_height(),
                 canvas.tostring_rgb())
    name = folder + '/gen_frame'+'{0:04d}'.format(i)+'.png'
    pil_image.save(name,"PNG")
    plt.close("all")

    
import IPython
from IPython.display import Image

img_gif = create_gif_from_frames(folder)
Image(filename=img_gif)

StopIteration: 